## 1 - Downloading the Necessary YOLOv3 Files

In [2]:
import cv2
import numpy as np

# YOLO modelini yükle
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getUnconnectedOutLayersNames()

# Resmi aç
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Örneğin, genişlik 640 piksel
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)  # Örneğin, yükseklik 480 piksel

# Resmi YOLO ile işleme
def predict(img):
    height, width, _ = img.shape
    blob = cv2.dnn.blobFromImage(img, scalefactor = 0.00392, size = (416, 416), mean = (0, 0, 0), swapRB = True, crop=False)
    net.setInput(blob)
    outs = net.forward(layer_names)

    
    # Tespit sonuçlarını işleme ve görselleştirme
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Belirli bir güvenilirlik eşiği
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # Non-Max Suppression uygula
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # NMS sonuçlarını görselleştirme
    font = cv2.FONT_HERSHEY_SIMPLEX
    for i in range(len(boxes)):
        if i in indices:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0)  # Yeşil renk
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, f'{label} {confidence:.2f}', (x, y - 10), font, 0.5, color, 2)
            
    return img

# Sonucu göster

while True:
    ret,frame = cap.read()
    cv2.imshow('Our live Sketcher',predict(frame))
    if cv2.waitKey(1) == 13:  #enter
        break


cap.release()
cv2.destroyAllWindows()
print('Done!!!')

Done!!!
